# Sentence embeddings with Python

In this section, we will explore more advanced embedding models that are based on the transformers neural network architecture. They also use tokens, semantic units smaller than words, that carries more semantic information than words. Theses models are also able to compute embedding for an entire piece of texte (like a sentence).

## Setup

Please check you have configured your environment properly with uv (see [setup](../setup.md))

## Load required libraries

In [1]:
from fastembed import TextEmbedding
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize

/home/pierre/workp/teaching/BI_M2_LLM/llm-practicals.git/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load the model

This model is a reduced version of the model [`nomic-embed-text-v-1.5`](https://huggingface.co/nomic-ai/nomic-embed-text-v1.5). It is a 137-million parameter model that is trained on a large corpus of text to generate embeddings for sentences. Generated embeddings are 768-dimensional vectors that capture the semantic meaning of the input text.

Loading the model can take a few minutes, so be patient.

In [2]:
model = TextEmbedding("nomic-ai/nomic-embed-text-v1.5-Q")

/tmp/ipykernel_687068/8704592.py:1: UserWarning: The model 'nomic-ai/nomic-embed-text-v1.5-Q' has been updated on HuggingFace. Please review the latest documentation on HF and release notes to ensure compatibility with your workflow. 
  model = TextEmbedding("nomic-ai/nomic-embed-text-v1.5-Q")


## Explore sentence embeddings

We will first compute the embedding vector for a sentence:

In [3]:
# The .embed() method returns a generator that we convert to a list
# and then take the first element to get the embedding vector.
vector = list(model.embed("Hello world!"))[0]  

Display the embedding vector and its size.

In [4]:
print("Ten first elements of the vector:")
print(vector[:10])
print(f"\nVector size: {len(vector)}")

Ten first elements of the vector:
[ 0.06642091  0.20711488 -4.30757713 -0.06292436 -0.65036206  1.85833189
 -0.11560041 -0.24147129 -0.5290895  -1.38232353]

Vector size: 768


With this model, embeddings are 768-dimensional vectors.

We can also compute the embedding for a list of sentences. The model will return a list of embeddings, one for each sentence.

In [5]:
sentences = [
    "The weather is lovely today.",
    "It's very sunny outside!",
    "Are you watching the soccer game on TV?",
    "I love playing football.",
    "Are you studying nanoporous materials?",
    "Physics is the science of matter and energy."
]
embeddings = np.array(list(model.embed(sentences)))
print("\nFive first elements of the embedding vectors for each sentence:")
print(embeddings[:, :5])
print(f"Shape of embedding vectors: {embeddings.shape}")


Five first elements of the embedding vectors for each sentence:
[[ 0.40734354 -0.12720139 -4.44438326 -0.22155395  0.56882895]
 [-0.13132822  0.18084796 -4.59579621  0.13161762 -0.6519796 ]
 [ 1.35002802  1.58990231 -3.40088714  0.38871316  0.80267872]
 [ 0.27677379  1.24250409 -4.05348274  0.25858318  0.99779707]
 [ 1.11604745  2.28325894 -2.88656797 -0.21893531  0.68955154]
 [ 1.77145601  2.12199622 -3.45457127 -0.72396081  1.12455912]]
Shape of embedding vectors: (6, 768)


In this example, we have 6 vectors with 768 dimensions.

## Compare sentences

We can compare the embeddings of two sentences by computing the cosine similarity between corresponding vectors.
To speed up the computation, we will first normalize the embeddings.

Nomalization and cosine similarity can be computed with the `scikit-learn` library or with the `numpy` library.

### Method 1 with scikit-learn

In [6]:
# Normalize embeddings
embeddings_normalized = normalize(embeddings, axis=1)
# Compute the cosine similarity between embeddings
similarity_matrix = cosine_similarity(embeddings_normalized, embeddings_normalized)
print(similarity_matrix)

[[1.         0.7519107  0.45709374 0.48242015 0.35106745 0.45631191]
 [0.7519107  1.         0.53255499 0.49246394 0.34660804 0.43519417]
 [0.45709374 0.53255499 1.         0.5587763  0.43619317 0.42865404]
 [0.48242015 0.49246394 0.5587763  1.         0.34444231 0.39961487]
 [0.35106745 0.34660804 0.43619317 0.34444231 1.         0.54134965]
 [0.45631191 0.43519417 0.42865404 0.39961487 0.54134965 1.        ]]


### Method 2 with Numpy

Cosine similarity between vectors X and Y is the normalized dot product of X and Y:

In [7]:
# Normalize embeddings
norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
embeddings_normalized = embeddings / norms
# Compute cosine similarity matrix
similarity_matrix = np.dot(embeddings_normalized, embeddings_normalized.T)
print(similarity_matrix)

[[1.         0.7519107  0.45709374 0.48242015 0.35106745 0.45631191]
 [0.7519107  1.         0.53255499 0.49246394 0.34660804 0.43519417]
 [0.45709374 0.53255499 1.         0.5587763  0.43619317 0.42865404]
 [0.48242015 0.49246394 0.5587763  1.         0.34444231 0.39961487]
 [0.35106745 0.34660804 0.43619317 0.34444231 1.         0.54134965]
 [0.45631191 0.43519417 0.42865404 0.39961487 0.54134965 1.        ]]


### Find most similar sentences

In [8]:
# Set the diagonal to -inf to exclude self-similarity.
np.fill_diagonal(similarity_matrix, -np.inf)
# Find the most similar sentence for each sentence.
for i, sentence in enumerate(sentences):
    row = similarity_matrix[i]
    most_sim_idx = np.argmax(row)
    print(f"Target sentence      : {sentence}")
    print(f"Most similar sentence: {sentences[most_sim_idx]}")
    print(f"Cosine similarity    : {row[most_sim_idx]:.3f}")
    print("-" * 70)

Target sentence      : The weather is lovely today.
Most similar sentence: It's very sunny outside!
Cosine similarity    : 0.752
----------------------------------------------------------------------
Target sentence      : It's very sunny outside!
Most similar sentence: The weather is lovely today.
Cosine similarity    : 0.752
----------------------------------------------------------------------
Target sentence      : Are you watching the soccer game on TV?
Most similar sentence: I love playing football.
Cosine similarity    : 0.559
----------------------------------------------------------------------
Target sentence      : I love playing football.
Most similar sentence: Are you watching the soccer game on TV?
Cosine similarity    : 0.559
----------------------------------------------------------------------
Target sentence      : Are you studying nanoporous materials?
Most similar sentence: Physics is the science of matter and energy.
Cosine similarity    : 0.541
-------------------